In [2]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_circles
from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.naive_bayes import GaussianNB as NB

# libraries to test against own implementation
from sklearn.svm import SVC
from sklearn.svm import LinearSVC as LSVC

cvxopt.solvers.options['show_progress'] = False
np.random.seed(760)

In [3]:
## Linear Logistic Regression Class
class LinLog():
  def __init__(self, epoch=100, rate=1e-3):
    self.epoch = epoch
    self.rate = rate
    self.w = None
    self.b = None

  def sigmoid(self,x, der=False):
    return 1 / (1+np.exp(-x))

  def fit(self, x, y):
    weights = np.zeros(x.shape[1])
    b_val = 0
    n = len(y)
    y = (y+1)/2
    for i in range(1,self.epoch+1):
      for j, x_train in enumerate(x):
        sig = self.sigmoid(np.dot(x_train,weights)+b_val)
        weights = weights - (self.rate*x_train*(sig-y[j]))/n
        b_val = b_val - (self.rate*(sig-y[j]))/n
    self.w = weights
    self.b = b_val

  def predict(self, x):
    return np.sign(np.dot(x,self.w)+self.b)

In [4]:
## Linear Support Vector Machine Class
class LinearSVM():
  def __init__(self, epoch=1000, rate=.01, lamb=1000, c=0):
    self.epoch = epoch
    self.rate = rate
    self.lamb = lamb
    self.c = c
    self.w = None
    self.b = None


  def fit(self,x,y):
    weight = np.zeros(x.shape[1])
    b_val = 0
    for epoch in range(1, self.epoch+1):
      for i, x_test in enumerate(x):
        y_pred = np.dot(x_test,weight)
        if (y[i]*y_pred < 1-self.c):
          weight = weight + self.rate*((y[i]*x_test)-(2*weight/self.lamb))
          b_val = b_val - self.rate*y[i]
        else:
          weight = weight + self.rate*(-2*weight/self.lamb)
      weight = weight/len(x)
      b_val = b_val/len(x)
    self.w = weight
    self.b = b_val

  def predict(self, x):
    return np.sign(np.dot(x,self.w)+self.b)



In [5]:
## Kernel Support Vector Machine Class
class KernelSVM():
  def __init__(self, c, kernel='linear', degree=2, sigma=1):
    if kernel == 'linear':
      self.kernel = self.linear_kernel
      self.prefix = 'Linear'
    elif kernel == 'poly':
      self.kernel = self.poly_kernel
      self.degree = degree
      self.prefix = 'Polynomial'
    else:
      self.kernel = self.gauss_kernel
      self.sigma = sigma
      self.prefix = 'Gaussian'
    self.c = float(c)
    self.w = None
    self.b = None


  def linear_kernel(self, x1, x2):
    return np.dot(x1,x2)

  def poly_kernel(self, x1, x2):
    return (1+np.dot(x1,x2))**self.degree

  def gauss_kernel(self, x1, x2):
    return np.exp(-linalg.norm(x1-x2)**2 / (2*(self.sigma**2)))

  def fit(self, x, y):
    y = y.reshape(-1,1)
    n, features = x.shape

    # calculate kernel elements
    K = np.zeros((n,n))
    for i in range(n):
      for j in range(n):
        K[i,j] = self.kernel(x[i],x[j])

    # dual can be rewritten to quadratic so we use quadratic solver
    # add small values to diagonal to ensure positive definite
    P = cvxopt.matrix((np.outer(y,y) * K)+1e-3*np.eye(len(y)))
    q = cvxopt.matrix(np.ones(n) * -1)

    # 0 <= alpha <= C
    G = cvxopt.matrix(np.vstack((-1*np.eye(n),np.eye(n))))
    h = cvxopt.matrix(np.hstack((np.zeros(n),np.ones(n)*self.c)))

    # Y.T @ alpha = 0
    A = cvxopt.matrix(y, (1,n),tc='d')
    b = cvxopt.matrix(0.0)

    # solve the equation using the given bounds
    sol = cvxopt.solvers.qp(P, q, G, h, A, b)
    alph = np.array(sol['x'])

    # support vectors will correspond to nonzero elements
    epsilon = 1e-5
    sv = (alph > epsilon).flatten()
    self.a = alph[sv]
    self.sv_x = x[sv]
    self.sv_y = y[sv]

    # b = A*Y
    self.b = 0.0
    for i in range(len(self.a)):
      self.b += self.sv_y[i]
      for j in range(len(self.a)):
        self.b -= self.a[j]*self.sv_y[j]*self.kernel(self.sv_x[i],self.sv_x[j])
    self.b /= len(self.a)

    # w = A*Y*X
    if self.kernel == self.linear_kernel:
      self.w = np.zeros(features)
      for i in range(len(self.a)):
        self.w += self.a[i]*self.sv_y[i]*self.sv_x[i]
    else:
      self.w = None # if nonlinear we cannot calculate weights for hyperplane
    return

  def predict(self, x):
    if self.w is not None:
      # y = w^T.x+b
      return np.sign(np.dot(x,self.w)+self.b)
    else:
      # y = sum(A*Y*K(x,x'))+b
      y_pred = np.zeros(len(x))
      for i in range(len(x)):
        temp = 0
        for j in range(len(self.a)):
          temp += self.a[j]*self.sv_y[j]*self.kernel(x[i], self.sv_x[j])
        y_pred[i] = temp
      return np.sign(y_pred+self.b)

In [6]:
## Kernel Logistic Regression Class
class KernLR():
  def __init__(self, c=1e-4, iter=10, rate=1e-3, kernel='linear', degree=2, sigma=1.0):
    if kernel == 'linear':
      self.kernel = self.linear_kernel
      self.prefix = 'Linear'
    elif kernel == 'poly':
      self.kernel = self.poly_kernel
      self.degree = degree
      self.prefix = 'Polynomial'
    else:
      self.kernel = self.gauss_kernel
      self.sigma = sigma
      self.prefix = 'Gaussian'
    self.c = float(c)
    self.iter = iter
    self.rate = rate
    self.x = None
    self.y = None

  def linear_kernel(self, x1, x2):
    return np.dot(x1,x2.T)

  def poly_kernel(self, x1, x2):
    return (1+np.dot(x1,x2.T))**self.degree

  def gauss_kernel(self, x1, x2):
    return np.exp(-linalg.norm(x1-x2)**2 / (2*(self.sigma**2)))

  def fit(self, x, y):
    self.x = x
    self.y = y.reshape(-1,1)
    self.alph = np.zeros((x.shape[0],1))
    self.k = self.kernel(x,x)

    for i in range(self.iter):
      self.alph = self.alph - self.rate*(np.dot(self.k,y) + self.c * np.dot(self.k, self.alph))

  def predict(self, x):
    print((self.y*self.alph).shape)
    print(self.kernel(self.x,x).shape)
    y_pred = np.array(np.sign(1/(1 + np.exp(-np.dot(self.y*self.alph, self.kernel(self.x,x))))))
    return y_pred

In [7]:
## Neural Network Class
class NeuralNet():
  def __init__(self, epoch, batch, input_size=2, verbose=2):
    self.epoch = epoch
    self.batch = batch
    self.verbose = verbose
    self.input_size = input_size
    self.model = self.init_model()

  def init_model(self):
    model = keras.models.Sequential([
                        keras.layers.Dense(64, activation='relu', input_shape=(self.input_size,)),
                        keras.layers.Dense(2, activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.Adam(),loss=keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])
    return model
  
  def fit(self, x, y):
    y = keras.utils.to_categorical((y+1)/2)
    self.model.fit(x,y,batch_size=self.batch, epochs=self.epoch, verbose=self.verbose)

  def predict(self, x):
    return -((np.argmin(self.model.predict(x),axis=1)*2)-1)

In [8]:
## Methods to generate datasets and to plot decision boundaries
def gen_rand(mu, sig):
  x0 = np.random.normal(0,sig,size=(750,2))
  x0[:,0] += mu
  x1 = np.random.normal(0,sig,size=(750,2))
  x1[:,0] -= mu
  x = np.vstack((x0,x1))
  y0 = np.ones((750,1))
  y1 = -1*np.ones((750,1))
  y = np.vstack((y0,y1))
  x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.33, stratify=y)
  x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, stratify=y_temp)
  return x_train, y_train, x_test, y_test, x_val, y_val

def gen_circ(num=200, factor=0.8, scale=2, noise=None):
  x,y = make_circles(n_samples=num, factor=factor, noise=noise)
  x = x*scale
  y = ((y*2)-1).reshape(-1,1)
  x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.33, stratify=y)
  x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, stratify=y_temp)
  return x_train, y_train, x_test, y_test, x_val, y_val

def gen_canc_data():
  data = load_breast_cancer()
  x = data.data
  y = (data.target*2)-1
  x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.33, stratify=y)
  x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, stratify=y_temp)
  return x_train, y_train, x_test, y_test, x_val, y_val


def plot_boundary(val,lab,clf,rang,num=100,title=None,nn=False):
  x0g = np.linspace(-rang,rang,num)
  x1g = np.linspace(-rang,rang,num)
  dec = np.zeros((num,num))
  if nn:
    inputs = np.zeros((num*num,2))
    for i in range(num):
      for j in range(num):
        inputs[i*num+j] = np.array([[x0g[j],x1g[i]]])
    dec = clf.predict(inputs)
    dec = dec.reshape(num,num)
    
  else:
    for i, x0 in enumerate(x0g):
      for j, x1 in enumerate(x1g):
        x = np.array([[x0,x1]])
        dec[j,i] = np.sign(clf.predict(x))
  plt.contourf(x0g,x1g,dec,colors=['blue','red'],alpha=0.5)
  plt.scatter(val[:,0],val[:,1],c=lab,cmap='bwr')
  if title is not None:
    plt.title(title)
  plt.show(block=False)

In [9]:
# Test for each of the classifiers with parameters allowing different test functions

def test_linlog(x_train, y_train, x_test, y_test, x_val, y_val,plot=True):
  log_clf = LinLog(epoch=1000)
  log_clf.fit(x_train,y_train)
  y_pred = log_clf.predict(x_test)
  err = np.mean(y_pred.flatten() != y_test.flatten())
  if plot:
    plot_boundary(x_test,y_test,log_clf,6,200,"Linear Log Reg Boundary w/ Test Points")
    print("Linear Log Reg Test Error Rate: %f" %(err*100),flush=True)
  return err

def test_linsvm(x_train, y_train, x_test, y_test, x_val, y_val,plot=True):
  lin_clf = LinearSVM(epoch=1000)
  lin_clf.fit(x_train,y_train)
  y_pred = lin_clf.predict(x_test)
  err = np.mean(y_pred.flatten() != y_test.flatten())
  if plot:
    plot_boundary(x_test,y_test,lin_clf,6,200,"Linear SVM Boundary w/ Test Points")
    print("Linear SVM Test Error Rate: %f" %(err*100),flush=True)
  return err

def test_svm(x_train, y_train, x_test, y_test, x_val, y_val,plot=True,kernel='linear',degree=2,sigma=1):
  c_vals = np.logspace(-3,5,9)
  errs = []
  for c in c_vals:
    svm_clf = KernelSVM(c=c,kernel=kernel,degree=degree,sigma=sigma)
    svm_clf.fit(x_train,y_train)
    y_pred = svm_clf.predict(x_val)
    errs.append(np.mean(y_pred.flatten() != y_val.flatten()))
  c_opt = c_vals[np.argmin(errs)]
  svm_clf = KernelSVM(c=c_opt,kernel=kernel,degree=degree,sigma=sigma)
  svm_clf.fit(x_train,y_train)
  y_pred = svm_clf.predict(x_test)
  err = np.mean(y_pred.flatten() != y_test.flatten())
  if plot:
    plot_boundary(x_test,y_test,svm_clf,6,100,"%s SVM Boundary w/ Test Points" %(svm_clf.prefix))
    print("SVM Test Error Rate: %f" %(err*100), flush=True)
  return err, c_opt

def test_svc(x_train, y_train, x_test, y_test, x_val, y_val,plot=True,kernel='linear',degree=2):
  c_vals = np.logspace(-3,5,9)
  errs = []
  for c in c_vals:
    svc_clf = SVC(C=c, kernel=kernel, degree=degree)
    svc_clf.fit(x_train,y_train.flatten())
    y_pred = svc_clf.predict(x_val)
    errs.append(np.mean(y_pred.flatten() != y_val.flatten()))
  c_opt = c_vals[np.argmin(errs)]
  svc_clf = SVC(C=c_opt, kernel=kernel, degree=degree)
  svc_clf.fit(x_train,y_train.flatten())
  y_pred = svc_clf.predict(x_test)
  err = np.mean(y_pred.flatten() != y_test.flatten())
  if plot:
    plot_boundary(x_test,y_test,svc_clf,6,100,"Poly SVM Boundary w/ Test Points")
    print("SVM Test Error Rate: %f" %(err*100), flush=True)
  return err, c_opt

def test_ker(x_train, y_train, x_test, y_test, x_val, y_val,plot=True,kernel='linear',degree=2,sigma=1):
  c_vals = np.logspace(-9,-3,7)
  errs = []
  for c in c_vals:
    log_clf = KernLR(c=c,kernel=kernel,degree=degree,sigma=sigma)
    log_clf.fit(x_train, y_train)  
    y_pred = log_clf.predict(x_val)
    errs.append(np.mean(y_pred.flatten() != y_val.flatten()))
  c_opt = c_vals[np.argmin(errs)]
  log_clf = KernLR(c=c_opt,kernel=kernel,degree=degree,sigma=sigma)
  log_clf.fit(x_train,y_train)
  y_pred = log_clf.predict(x_test)
  err = np.mean(y_pred.flatten() != y_test.flatten())
  if plot:
    plot_boundary(x_test,y_test,log_clf,6,100,"%s SVM Boundary w/ Test Points" %(log_clf.prefix))
    print("SVM Test Error Rate: %f" %(err*100), flush=True)
  return err, c_opt

def test_knn(x_train, y_train, x_test, y_test, x_val, y_val,plot=True):
  k_vals = np.array([1,3,5,9])
  errs = []
  for k in k_vals:
    knn_clf = KNN(n_neighbors=k)
    knn_clf.fit(x_train,y_train.squeeze())
    y_pred = knn_clf.predict(x_val)
    errs.append(np.mean(y_pred != y_val.squeeze()))
  k_opt = k_vals[np.argmin(errs)]
  knn_clf = KNN(n_neighbors=k_opt)
  knn_clf.fit(x_train,y_train.squeeze())
  err = 1-knn_clf.score(x_test,y_test)
  if plot:
    print("Optimal k-value: %d" %(k_opt), flush=True)
    plot_boundary(x_test,y_test,knn_clf,6,100,"KNN Boundary w/ Test Points")
    print("KNN Test Error Rate: %f" %(100*err))
  return err, k_opt

def test_nb(x_train, y_train, x_test, y_test, x_val, y_val,plot=True):
  nb_clf = NB()
  nb_clf.fit(x_train,y_train.squeeze())
  y_pred = nb_clf.predict(x_test)
  err = np.mean(y_pred != y_test.squeeze())
  if plot:
    plot_boundary(x_test, y_test, nb_clf, 6, 100, "NB Boundary w/ Test Points")
    print("NB Test Error Rate: %f" %(err*100),flush=True)
  return err

def test_nn(x_train, y_train, x_test, y_test, x_val, y_val,plot=True, input_size=2):
  nn_clf = NeuralNet(epoch=50, batch=10, input_size=input_size, verbose=0)
  nn_clf.fit(x_train,y_train)
  y_pred = nn_clf.predict(x_test)
  err = np.mean(y_pred.flatten() != y_test.flatten())
  if plot:
    nn_clf.model.summary()
    plot_boundary(x_test, y_test, nn_clf, 6, 100, "Neural Net Boundary w/ Test Points", nn=True)
    print("Neural Net Test Error Rate %f" %(err*100),flush=True)
  return err

### Question 2.2.1

In [10]:
# working with linear dataset. uncomment tests to run

mu = 2.5
sig = 1
params = gen_rand(mu,sig)

#test_linlog(*params);
#test_svm(*params,kernel='linear');
#test_knn(*params);
#test_nb(*params);
#test_nn(*params);

In [11]:
# running similar tests as section above but with varying values of mu

mu_vals = np.arange(1.0,2.2,0.2)
sig = 1
plot = False

linlog_errs = []
linsvm_errs = []
knn_errs = []
nb_errs = []

#for mu in mu_vals:
#  params = gen_rand(mu,sig)
#  linlog_errs.append(test_linlog(*params,False))
#  linsvm_errs.append(test_svm(*params,False,kernel='linear')[0])
#  knn_errs.append(test_knn(*params,False)[0])
#  nb_errs.append(test_nb(*params,False))

In [12]:
if plot:
  plt.figure()
  plt.plot(mu_vals,linlog_errs,label='LinLog')
  plt.plot(mu_vals,linsvm_errs,label='LinSVM')
  plt.plot(mu_vals,knn_errs,label='KNN')
  plt.plot(mu_vals,nb_errs,label='Naive Bayes')
  plt.legend()
  plt.title("Error Rates for Various Classifiers")
  plt.ylabel("Error Rate")
  plt.xlabel("mu Value");

### Question 2.2.2

In [13]:
# working with nonlinear (circles) dataset. uncomment tests to run

params_nonlin = gen_circ(num=1500,scale=5)

#test_linlog(*params_nonlin);
#test_svm(*params_nonlin,kernel='linear');
#test_svm(*params_nonlin,kernel='rbf');
#test_ker(*params_nonlin);
#test_knn(*params_nonlin);
#test_nn(*params_nonlin);

### Question 2.3

In [14]:
# working with cancer dataset. uncomment tests to run

canc_params = gen_canc_data()

#print(test_knn(*canc_params,plot=False));
#print(test_linlog(*canc_params,plot=False));
#print(test_svm(*canc_params,plot=False,kernel='linear'));
#print(test_nn(*canc_params,input_size=30,plot=False));
#print(test_svc(*canc_params,plot=False,kernel='poly',degree=2));
#print(test_svc(*canc_params,plot=False,kernel='poly',degree=3));
#print(test_svc(*canc_params,plot=False,kernel='poly',degree=4));
#print(test_svc(*canc_params,plot=False,kernel='rbf'));

In [15]:
# L1 regualarized linear SVM to extract important features

labels = load_breast_cancer().feature_names
errs = []
c_vals = np.logspace(-3,5,9)
for c in c_vals:
  lsvc = LSVC(penalty='l1',dual=False,C=c,max_iter=100000)
  lsvc.fit(canc_params[0],canc_params[1])
  y_pred = lsvc.predict(canc_params[4])
  err = np.mean(y_pred != canc_params[5])
  errs.append(err)
c_opt = c_vals[np.argmin(errs)]
print(errs[np.argmin(errs)])
lsvc = LSVC(penalty='l1',dual=False,C=c_opt,max_iter=100000)
lsvc.fit(canc_params[0],canc_params[1])
tar = np.abs(lsvc.coef_) > 3
print(labels[tar.flatten()])

0.0
['mean concave points' 'mean symmetry' 'worst smoothness'
 'worst concave points' 'worst symmetry']
